# Ejercicio : Aplicando PySpark - Fuente de Datos SUNEDU


### Descargando fuente de datos "Universidades"

Descargamos fuente de datos universidades en formato .CSV del portal datosabiertos correspondiente a la SUNEDU.

URL: https://www.datosabiertos.gob.pe/dataset/sunedu-listado-de-universidades

Fecha Actualización de fuente de dato: 12/09/2019

### Cargamos fuente de datos "Universidades.csv" a nuestro programa

In [0]:
# Cargamos fichero universidaddes.csv a nuestro programa
file = "file:///home/gchacaltanab/datos/Universidades.csv"
data_csv = sc.textFile(file)

### Analizamos estructura de la fuente de datos "Universidades.csv"

In [0]:
# Obtenemos la cabecera del archivo para conocer su estructura.
header = data_csv.first()
header_names = str(header).split(',')

In [0]:
# Devolver estructura del archivo.
print("\033[1m ESTRUCTURA DEL ARCHIVO \033[0m")
list(header_names)

 ESTRUCTURA DEL ARCHIVO 


['CODIGO',
 'NOMBRE',
 'UNIV TIPO',
 'ESTADO LICENCIAMIENTO',
 'PERIODO LICENC',
 'DEPARTAMENTO',
 'PROVINCIA',
 'DISTRITO',
 'LATITUD',
 'LONGITUD']

### Eliminamos la cabecera del fichero

In [0]:
data_csv_body = data_csv.filter(lambda x: x!=header)

In [0]:
# Devolvemos la cantidad de registros que contiene el archivo.
print("El archivo contiene %s registros" % data_csv_body.count())

El archivo contiene 134 registros


In [0]:
# Imprimimos el primer registr, para conocer su estructura y contenido.
data_csv_body.take(1)

['001,UNIVERSIDAD NACIONAL MAYOR DE SAN MARCOS,PUBLICO,LICENCIADA,10,LIMA,LIMA,LIMA,-12.046374,-77.042793399999994']

In [0]:
# Creamos RDD para trabajar con la data necesaria para el análisis.
pre_data = data_csv_body.map(lambda x: x.split(','))
data = pre_data.map(lambda x: (x[1], x[2], x[3], x[4], x[5],x[6],x[7]))

In [0]:
# Visualizamos los dos primeros registros del RDD. Repamos la estructura de la fuente de datos.
data.take(2)

[('UNIVERSIDAD NACIONAL MAYOR DE SAN MARCOS',
  'PUBLICO',
  'LICENCIADA',
  '10',
  'LIMA',
  'LIMA',
  'LIMA'),
 ('UNIVERSIDAD NACIONAL DE SAN CRIST�BAL DE HUAMANGA',
  'PUBLICO',
  'LICENCIADA',
  '6',
  'AYACUCHO',
  'HUAMANGA',
  'AYACUCHO')]

### ¿Cuántos tipos de universidades existen?

In [0]:
# Creamos RDD que contenga los tipos de universidades añadiendole el valor de 1 para calcular cantidades.
universities_types = data.map(lambda x: (x[1],1))
universities_types.reduceByKey(lambda x,y: x+y).takeOrdered(3, lambda x: -x[1])

[('PRIVADA SOCIETARIA', 50), ('PUBLICO', 43), ('PRIVADA ASOCIATIVA', 41)]

### ¿Cuántas universidades se encuentrán licenciadas, en proceso o denegadas?

In [0]:
# Creamos RDD que contenga los tipos de licenciamiento añadiendole el valor de 1 para calcular cantidades.
universities_state_lic = data.map(lambda x: (x[2],1))
universities_state_lic.reduceByKey(lambda x,y: x+y).takeOrdered(3, lambda x: -x[1])

[('LICENCIADA', 69), ('EN PROCESO', 55), ('DENEGADA', 10)]

### ¿Cuántas universidades hay por departamento?

In [0]:
universities_by_department = data.map(lambda x: (x[4],1))
universities_by_department.reduceByKey(lambda x,y: x+y).takeOrdered(30, lambda x: -x[1])

[('LIMA', 53),
 ('LA LIBERTAD', 7),
 ('AREQUIPA', 7),
 ('JUNiN', 6),
 ('LAMBAYEQUE', 6),
 ('CUSCO', 5),
 ('LORETO', 5),
 ('PUNO', 4),
 ('CAJAMARCA', 4),
 ('ANCASH', 4),
 ('AYACUCHO', 3),
 ('ICA', 3),
 ('PIURA', 3),
 ('HUaNUCO', 3),
 ('TACNA', 3),
 ('UCAYALI', 3),
 ('APURiMAC', 3),
 ('CALLAO', 2),
 ('MOQUEGUA', 2),
 ('HUANCAVELICA', 2),
 ('AMAZONAS', 2),
 ('PASCO', 1),
 ('SAN MARTiN', 1),
 ('TUMBES', 1),
 ('MADRE DE DIOS', 1)]

### ¿Cuántas son las universidades privadas que están licenciadas?

In [0]:
report_priv_lic = data.filter(lambda x: (("PRIVADA SOCIETARIA" in x[1] or "PRIVADA ASOCIATIVA" in x[1]) and "LICENCIADA" in x[2]))
print("Son %s universidades privadas que están licenciadas. " % (report_priv_lic.count()))

Son 38 universidades privadas que están licenciadas. 


### ¿Cuántas son las universidades públicas que están en proceso de licenciamiento?

In [0]:
report_nac_pro = data.filter(lambda x: ("PUBLICO" in x[1] and "EN PROCESO" in x[2]))
print("Son %s universidades públicas que están en proceso de licenciamiento. " % (report_nac_pro.count()))

Son 12 universidades públicas que están en proceso de licenciamiento. 


In [0]:
print("UNIVERSIDADES PÚBLICAS EN PROCESO DE LICENCIAMIENTO")
report_nac_pro.map(lambda x: (x[0],x[4])).collect()

UNIVERSIDADES PÚBLICAS EN PROCESO DE LICENCIAMIENTO


[('UNIVERSIDAD NACIONAL SAN LUIS GONZAGA DE ICA', 'ICA'),
 ('UNIVERSIDAD NACIONAL FEDERICO VILLARREAL', 'LIMA'),
 ('UNIVERSIDAD NACIONAL AGRARIA DE LA SELVA', 'HUaNUCO'),
 ('UNIVERSIDAD NACIONAL HERMILIO VALDIZ�N DE HU�NUCO', 'HUaNUCO'),
 ('UNIVERSIDAD NACIONAL DE EDUCACI�N ENRIQUE GUZM�N Y VALLE', 'LIMA'),
 ('UNIVERSIDAD NACIONAL DANIEL ALC�DES CARRI�N', 'PASCO'),
 ('UNIVERSIDAD NACIONAL DEL CALLAO', 'CALLAO'),
 ('UNIVERSIDAD NACIONAL JOS� FAUSTINO S�NCHEZ CARRI�N', 'LIMA'),
 ('UNIVERSIDAD NACIONAL PEDRO RU�Z GALLO', 'LAMBAYEQUE'),
 ('UNIVERSIDAD NACIONAL DE TUMBES', 'TUMBES'),
 ('UNIVERSIDAD NACIONAL AMAZ�NICA DE MADRE DE DIOS', 'MADRE DE DIOS'),
 ('UNIVERSIDAD NACIONAL MICAELA BASTIDAS DE APUR�MAC', 'APURiMAC')]